# OCLC Parse Classify

This script expects to work on a TSV that has MARC XML in a `oclc_classify` column, likely added by the `download_oclc_classify` script previously.


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import json


In [ ]:
path_to_tsv = "/Users/m/Downloads/data-tmp/nyt_hardcover_fiction_bestsellers-hathitrust_metadata.tsv"
author_viaf = 'author_viaf'
author_lccn = 'author_lccn'
work_title = 'shorttitle' # or set to None
author_marc = 'author_marc' # the athuor name from the marc record
author_authorized_heading = 'author_authorized_heading'


# isbn_cache = json.load(open('/Users/m/Downloads/oclc_numbers.json'))



In [ ]:
def parse_classify_row(d):

    # do we have some classify data?
    if type(d['oclc_classify']) == str:   


        data = extract_classify(d['oclc_classify'])
        if data != None:

            # # the titles are too different on many levels, not a good check
            # # a little sanity check here, if we have the title, compare and make sure they look good
            # if work_title != None and pd.isnull(d[work_title]) == False:

            #     #which is shortest
            #     if len(data['work_title']) >=  len(str(d[work_title])):
            #         shortest = len(str(d[work_title]))
            #     else:
            #         shortest = len(data['work_title'])

            #     shorttitle = data['work_title'][0:shortest]
            #     shorttitle2 = str(d[work_title])[0:shortest]

            #     title_score = levenshtein(shorttitle,shorttitle2)
            #     if title_score > 20:
            #         print(shorttitle)
            #         print(shorttitle2)
            #         print('-------')

            # # this test showed lots of errors in the orginal data/marc record, nothing much more, not needed
            # if pd.isnull(d[author_viaf]) == True and data['work_author'] != None:
            #     if data['work_author']['viaf'] != None:

            #         #which is shortest
            #         if len(data['work_author']['name']) >=  len(str(d[author_marc])):
            #             shortest = len(str(d[author_marc]))
            #         else:
            #             shortest = len(data['work_author']['name'])


            #         shortauthor = data['work_author']['name'][0:shortest]
            #         shortauthor2 = str(d[author_marc])[0:shortest]


            #         if levenshtein(shortauthor,shortauthor2) >= shortest / 2:
            #             print("HERE HEREEHEH",data['work_author']['name'], "!=", d[author_marc], "|", d['oclc'] )

            # add in the author info if is missing
            if author_viaf not in d:
                d[author_viaf] = None
            if author_lccn not in d:
                d[author_lccn] = None   
                             
            if pd.isnull(d[author_viaf]) == True and data['work_author'] != None:
                if data['work_author']['viaf'] != None:
                    d[author_authorized_heading] = data['work_author']['name']
                    d[author_viaf] = data['work_author']['viaf']
           
            if pd.isnull(d[author_lccn]) == True and data['work_author'] != None:
                if data['work_author']['lccn'] != None:                    
                    d[author_lccn] = data['work_author']['lccn']
                    d[author_authorized_heading] = data['work_author']['name']
                    print(data['work_author'])


            
            # # d['oclc_largest_holding'] = data['largest_holding_oclc']
            # all_isbns = []
            # for e in data["editions"]:
            #     oclc = str(int(e['oclc']))
            #     if oclc in isbn_cache:
            #         if isbn_cache[oclc] != None:
                            
            #             cachedata = isbn_cache[oclc]
            #             if 'workExample' in cachedata:
            #                 if type(cachedata['workExample']) != list:
            #                     cachedata['workExample'] = [cachedata['workExample']]
                            
            #                 isbns = []
            #                 for url in cachedata['workExample']:
            #                     if 'isbn' in url:
            #                         isbns.append(url.split('/')[-1])
            #                 all_isbns = all_isbns + isbns
                            
            # all_isbns = list(set(all_isbns))
            # d['isbns'] = "|".join(all_isbns)

            d['oclc_eholdings'] = data['work_eholdings']
            d['oclc_holdings'] = data['work_holdings']
            d['oclc_owi'] = data['work_owi']
            

        else:
            #print("bad data", d['oclc_classify'])
            pass

        

    else:
        # print("No Classify data to parse:",d)
        pass
    
    return d

In [ ]:
def extract_classify(xml):


		soup = BeautifulSoup(str(xml))

		work_soup = soup.find("work")

		if work_soup == None:
			# print("can not parse xml:")
			# print(xml)
			return None

		results = {}

		results['work_statement_responsibility'] = None if work_soup.has_attr('author') == False else work_soup['author']
		results['work_editions'] = None if work_soup.has_attr('editions') == False else int(work_soup['editions'])
		results['work_eholdings'] = None if work_soup.has_attr('eholdings') == False else int(work_soup['eholdings'])
		results['work_format'] = None if work_soup.has_attr('format') == False else work_soup['format']
		results['work_holdings'] = None if work_soup.has_attr('holdings') == False else int(work_soup['holdings'])
		results['work_itemtype'] = None if work_soup.has_attr('itemtype') == False else work_soup['itemtype']
		results['work_owi'] = None if work_soup.has_attr('owi') == False else work_soup['owi']
		results['work_title'] = None if work_soup.has_attr('title') == False else work_soup['title']
		results['main_oclc'] = work_soup.text

		
		
		authors_soup = soup.find_all("author")
		results['authors'] = []
		for a in authors_soup:
			results['authors'].append({
					"name" : a.text.split('[')[0].strip(),
					"lccn" : None if a.has_attr('lc') == False else a['lc'],
					"viaf" : None if a.has_attr('viaf') == False else a['viaf']
				})

		for a in results['authors']:
			if a['lccn'] == 'null':
				a['lccn'] = None	
			if a['viaf'] == 'null':
				a['viaf'] = None	

		# try to find the first main contributor
		results['work_author'] = None
		if results['work_statement_responsibility'] != None:
			if len(results['work_statement_responsibility'].split("|"))>0:
				first_author = results['work_statement_responsibility'].split("|")[0].strip()
				for a in results['authors']:
					if a['name'].strip() == first_author:
						results['work_author'] = a



		results["normalized_ddc"] = None
		results["normalized_lcc"] = None

		ddc_soup = soup.find("ddc")
		if ddc_soup != None:
			ddc_soup = soup.find("ddc").find("mostpopular")
			if ddc_soup != None:
				if ddc_soup.has_attr('nsfa'):
					results["normalized_ddc"] = ddc_soup['nsfa']

		lcc_soup = soup.find("lcc")
		if lcc_soup != None:
			lcc_soup = soup.find("lcc").find("mostpopular")
			if lcc_soup != None:
				if lcc_soup.has_attr('nsfa'):
					results["normalized_lcc"] = lcc_soup['nsfa']


		results["headings"] = []
		heading_soup = soup.find_all("heading")
		for h in heading_soup:
			results["headings"].append({
					"id" : h['ident'],
					"src": h['src'],
					"value" : h.text
				})
			

		edition_soup = soup.find_all("edition")
		# print(isbn,len(edition_soup))
		results["editions"] = []
		for e in edition_soup:
			edition = {}
			edition['author'] = None if e.has_attr('author') == False else e['author']
			edition['eholdings'] = None if e.has_attr('eholdings') == False else int(e['eholdings'])
			edition['format'] = None if e.has_attr('format') == False else e['format']
			edition['holdings'] = None if e.has_attr('holdings') == False else int(e['holdings'])
			edition['itemtype'] = None if e.has_attr('itemtype') == False else e['itemtype']
			edition['language'] = None if e.has_attr('language') == False else e['language']
			edition['oclc'] = None if e.has_attr('oclc') == False else e['oclc']
			edition['title'] = None if e.has_attr('title') == False else e['title']
			results["editions"].append(edition)
		
		if len(results["editions"]) > 0:
			results["largest_holding_oclc"] = results["editions"][0]['oclc']

		return results

In [ ]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [ ]:
# load the tsv
df = pd.read_csv(path_to_tsv, sep='\t', header=0, low_memory=False)
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)

# run our function over all of them
df = df.apply(lambda d: parse_classify_row(d),axis=1 )  
# # overwrite back out
df.to_csv(path_to_tsv, sep='\t')

